# Import packages

In [4]:
from dotenv import load_dotenv
import os
import requests
import pandas as pd

In [34]:
# Load HF key
load_dotenv()
api_key = os.getenv("HF_API_KEY")

In [31]:
url = "https://api-inference.huggingface.co/models/mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"
headers = { "Authorization": f"Bearer {api_key}" }
payload = { "inputs": "How Britain can seize the next decade" } # "inputs" will accept single string or list of strings

response = requests.post(url, headers=headers, json=payload)
response.json()

[[{'label': 'positive', 'score': 0.6430162191390991},
  {'label': 'neutral', 'score': 0.355707585811615},
  {'label': 'negative', 'score': 0.001276212278753519}]]

In [28]:
ft_articles_df = pd.read_csv("ft_all_crypto.csv")

In [35]:
import pandas as pd
import requests
import time  # for rate-limiting

# Set up Hugging Face API

url = "https://api-inference.huggingface.co/models/mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"
headers = {"Authorization": f"Bearer {api_key}"}

# Define the sentiment analysis function
def get_sentiment(snippet):
    if not snippet or not isinstance(snippet, str):
        return "unknown"
    
    payload = {"inputs": snippet}
    try:
        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()  # raise exception for bad status
        result = response.json()
        if isinstance(result, list) and len(result) > 0:
            return result[0][0]['label']
    except Exception as e:
        print(f"Error processing: {snippet[:50]}... | Error: {e}")
        return "error"

# Apply to DataFrame
ft_articles_df['sentiment'] = ft_articles_df['snippet'].apply(lambda x: get_sentiment(x))

# Optional: if API rate limit is an issue, insert a delay
# ft_articles_df['sentiment'] = ft_articles_df['snippet'].apply(lambda x: (time.sleep(1), get_sentiment(x))[1])


Error processing: Penalty for Mike Novogratz’s group comes as SEC sc... | Error: 503 Server Error: Service Temporarily Unavailable for url: https://api-inference.huggingface.co/models/mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis
Error processing: But the craziness from the White House has not tur... | Error: 503 Server Error: Service Temporarily Unavailable for url: https://api-inference.huggingface.co/models/mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis
Error processing: FCA seeks to encourage more investment in shares a... | Error: 503 Server Error: Service Temporarily Unavailable for url: https://api-inference.huggingface.co/models/mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis
Error processing: The greenback’s dominance was forged on trade, all... | Error: 503 Server Error: Service Temporarily Unavailable for url: https://api-inference.huggingface.co/models/mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis


In [39]:
# filter df for articles where date is not null
ft_articles_df.isna().sum()

Unnamed: 0      0
title           0
link            0
snippet        15
date          145
symbol          0
sentiment       0
dtype: int64

In [38]:
ft_articles_df

,Unnamed: 0,title,link,snippet,date,symbol,sentiment
0,0,Trump goes for shock therapy,https://www.ft.comhttps://www.ft.com/content/f...,The market was far too optimistic,2025-04-03,BTC,positive
1,1,Bitcoin pares losses after sharp fall,https://www.ft.com/content/8c8062e5-2744-40ea-...,Bitcoin has recovered some ground after sharpl...,NaN,BTC,positive
2,2,Gold surges to record high as investors seek s...,https://www.ft.com/content/8c8062e5-2744-40ea-...,Gold rose to a new record high as Donald Trump...,NaN,BTC,positive
3,3,"Bitcoin falls below $85,000 after tariffs anno...",https://www.ft.com/content/8c8062e5-2744-40ea-...,The price of bitcoin sharply fell after Donald...,NaN,BTC,negative
4,4,Stablecoin operator Circle files for IPO as re...,https://www.ft.comhttps://www.ft.com/content/7...,US firm is latest crypto group to try to take ...,2025-04-02,BTC,positive
...,...,...,...,...,...,...,...
950,950,WPP cuts revenue forecast as US tech spending ...,https://www.ft.comhttps://www.ft.com/content/5...,Ad agency lowers annual guidance after technol...,2023-08-04,XRP,negative
951,951,WPP cuts growth forecast as North America reve...,https://www.ft.comhttps://www.ft.com/content/9...,WPP has lowered its growth forecasts for the y...,NaN,XRP,negative
952,952,Crypto got Rakoff’d,https://www.ft.comhttps://www.ft.com/content/5...,Enter the dragon,2023-08-03,XRP,neutral
953,953,X Corp threatens to sue digital hate researcher,https://www.ft.comhttps://www.ft.com/content/f...,Elon Musk’s X Corp has threatened to sue the C...,NaN,XRP,negative


In [40]:
ft_articles_df.groupby('sentiment').size()

sentiment
error         4
negative    136
neutral     549
positive    251
unknown      15
dtype: int64

# Daily Sentiment

In [ ]:
# converting sentiment to numerical values
sentiment_map = {"positive": 1, "neutral": 0, "negative": -1}
ft_articles_df['sentiment_score'] = ft_articles_df['sentiment'].map(sentiment_map)

# Average sentiment per day per crypto
daily_sentiment = ft_articles_df.groupby(['date', 'symbol'])['sentiment_score'].mean().reset_index()


In [42]:
daily_sentiment

,date,symbol,sentiment_score
0,2023-01-16,ETH,-1.0
1,2023-01-18,ETH,0.5
2,2023-01-20,ETH,-0.5
3,2023-01-25,ETH,1.0
4,2023-01-31,ETH,0.0
...,...,...,...
637,2025-04-02,BTC,0.5
638,2025-04-02,ETH,0.0
639,2025-04-02,USDT,0.5
640,2025-04-03,BTC,1.0
